In [ ]:
!pip install sodapy

In [ ]:
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)
from google.colab import userdata
token = userdata.get('token_app')
codigo_dataset = userdata.get('codigo_dataset')
# Example authenticated client (needed for non-public datasets):
client = Socrata("www.datos.gov.co",
                  str(token),timeout=1000)

In [ ]:
import time
# 3. Parámetros de paginación
limit = 500000
offset = 0
reintentos=5

lista_DF = []

print(f"Iniciando carga por lotes para el dataset: {codigo_dataset}")

# 4. Bucle para obtener y cargar los datos por lotes
while True:
    intentos = 0

    while intentos < reintentos:
        try:
            print(f"Obteniendo y cargando lote {offset}...")
            # Construye y ejecuta la consulta para el lote actual
            query = f"SELECT numero_del_contrato, numero_de_proceso, nit_de_la_entidad, documento_proveedor, estado_del_proceso LIMIT {limit} OFFSET {offset}"
            results = client.get(codigo_dataset, query=query)

            # Si la API no devuelve más registros, se termina el bucle
            if not results:
                print("Carga de datos finalizada.")
                break

            # Convierte el lote a un DataFrame de Spark y lo escribe en la tabla Delta
            df_temp=pd.DataFrame(results)
            df_temp.to_csv(f"/content/drive/MyDrive/secop/data_{offset}.csv", index=False)

            lista_DF.append(df_temp)

            print(f"Lote de {len(results)} registros desde offset {offset} cargado.")
            offset += limit
            break

        except Exception as e:
            intentos +=1
            print("Error al obtener o cargar el lote:", e)
            print(f"Intento {intentos} de {reintentos}...")
            time.sleep(40)
    else:
        print("Se alcanzó el número máximo de intentos. Terminando la carga.")
        break


Iniciando carga por lotes para el dataset: rpmr-utcd
Obteniendo y cargando lote 0...
Lote de 500000 registros desde offset 0 cargado.
Obteniendo y cargando lote 500000...
Lote de 500000 registros desde offset 500000 cargado.
Obteniendo y cargando lote 1000000...
Lote de 500000 registros desde offset 1000000 cargado.
Obteniendo y cargando lote 1500000...


KeyboardInterrupt: 

In [ ]:
DF_secop_id=pd.concat(lista_DF)

In [ ]:
import hashlib
import sqlite3

DF_secop_id['concatenar']=DF_secop_id['numero_de_proceso']+"||"+DF_secop_id['numero_del_contrato']+"||"+DF_secop_id['nit_de_la_entidad']+"||"+DF_secop_id['documento_proveedor']+"||"+DF_secop_id['numero_de_proceso']+"||"+DF_secop_id['numero_del_contrato']
DF_secop_id['id_unico'] = DF_secop_id['concatenar'].astype(str).apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest())
DF_secop_id.to_sql('secop_id', sqlite3.connect('/content/drive/MyDrive/secop/secop_id.db'), if_exists='replace', index=False)



1500000

In [ ]:
]